# Project 4: Sequential Structure from Motion

### Due 4/3/2019

### Graduate Students: Our next reading is [Snavely, 2006](http://195.130.87.21:8080/dspace/bitstream/123456789/636/1/Photo%20tourism%20exploring%20photo%20collections%20in%203D.pdf).  We'll have a written report on this one: these methods papers aren't as good for discussions as I'd hoped.

## Problem Statement

You have now developed code that takes two photographs, finds matching locations in both, determines the relative motion between the cameras that took both photos, and solves for the 3D position of those points using epipolar geometry.  **The next (and final for our purposes) stage is to extend this analysis to more than two images**, such that we can build 3D models of objects on the ground with just about as much detail as we'd like.

## Adding the third photo
How do we add these additional photos?  To be perfectly honest, at this point it's mostly an exercise in match housekeeping: we've already developed most of the code that we need.  First, let's consider what we've got after matching our first two images, $I_1$ and $I_2$.  First, we have a set of keypoints in each image, associated with a set of matches.  These matches have been quality controlled twice:  first by the ratio test, then by RANSAC in conjunction with the recovery of the essential matrix.  Assuming that we've used our known camera matrix to convert our pixel-wise coordinates to generalized coordinates, let's call these keypoints $\mathbf{x}_1$ and $\mathbf{x}_2$.  In practice, we can drop all of those keypoints for which there is not an associated accepted match.  Then, for each of our kept matches, we have the essential matrix $E_{12}$, from which we can extract a pair of projection matrices $\mathbf{P}_1 = [\mathbf{I}|\mathbf{0}]$ and $\mathbf{P}_2 = [\mathbf{R}_2|\mathbf{t}_2]$.  Using these projection matrices, we generated 3D, world coordinate location of the corresponding features that showed up robustly in both images.  We'll call these coordinates $\mathbf{X}_{12}$.  

To add a third image $\mathbf{I}_3$ to the mix, consider that the situation outlined above is sort of analogous to the information that we have when we want to do pose estimation with ground control points: we have 3D world coordinates as well as the image coordinates of a set of points (a bunch of them, usually!), and we want to recover the camera pose.  The problem is that the feature generalized coordinates that we've computed are for $I_1$ and $I_2$, but not $I_3$.  Is this a big problem?  Of course not!  We can simply find $\mathbf{x}_3$ in $I_3$ that correspond to $\mathbf{x}_2$, the keypoints in the second image.  Then we identify these keypoints with the 3D poi nts $\mathbf{X}_{12}$.  Thus we have image coordinates of features in the third image and the corresponding world coordinates: we can now perform pose estimation, just as we did in Project 1.  

Of course there are a few minor caveats: first, we need to filter out spurious matches between $\mathbf{x}_2$ and $\mathbf{x}_3$.  To do this, we can utilize a tool that we already have: RANSAC estimation of the essential matrix.  Because $I_2$ and $I_3$ are related by epipolar geometry just as $I_1$ and $I_2$ are, we can use the same subroutine to compute the essential matrix $\mathbf{E}_{23}$, and (critically) identify and filter outliers, i.e. we'll discard matches that don't don't correspond to the consensus view of the essential matrix.  This also leads to the next caveat, namely that we need an initial guess (call it $P_3^0$) for pose estimation to converge properly.  Where should this initial guess come from?  The $\mathbf{E}_{23}$ provides a rotation given as if camera 2 were canonical, i.e. $\mathbf{P_2'}=[\mathbf{I}|\mathbf{0}]$, $\mathbf{P_3}'=[\mathbf{R}_3'|\mathbf{t}_3']$.  We'll call it $P_3'$.  We need to convert this projection matrix to a coordinate system in which $I_1$ (not $I_2$) is canonical.  Fortunately, this is easy:
$$
P_3 \approx P_2 P_3'.
$$
This $P_3$, is a an excellent initial guess for pose estimation (in principle, it's rotation matrix should actually be correct).  Note that the translation component is only good up to a constant: however, this isn't too problematic because its direction is close to correct, and any optimization just needs to perform what amounts to a line search (a univariate optimization problem) to find the correct scaling. 

Once we have a robust estimation of the third camera's pose, we can use it do point triangulation on the correspondences between $I_2$ and $I_3$ not associated with an already-known world coordinate point, which allows us to augment our 3D model with new points.  Additionally, we can perform triangulation with *3 views*, potentially improving our accuracy.  Moreover, we can apply the process above iteratively, adding more and more images to generate a highly featured 3D model from (for example) 360 degrees worth of view angles.  

## Application
**Generate code that performs the above process for a third image.  Apply it to one of the 3D image datasets that we generated in class.  Note that we will be collecting aerial imagery from drones as well.  Apply this method to a sequence of drone imagery as well.**  As a challenge, can you implement code that sequentially adds an arbitrary number of images?


In [37]:
# Apply code to get keypoint matches
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
import numpy as np
import cv2
import piexif


def plot_pcd(pcd, title="", marker=".", s=3):
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111, projection='3d', aspect="equal")
    plt.title(title)
    ax.scatter(pcd[:, 0], pcd[:, 1], pcd[:, 2], c=pcd[:, 3:], marker=marker, s=s)
    plt.show()
    

def extract_pixel_colors(img, uv):
    return np.array([img[int(uvi[1]), int(uvi[0])] for uvi in uv]) / 255
    
    
def triangulate(P0, P1, u1, u2):
    X = [_triangulate(P0, P1, u1[i], u2[i]) for i in range(len(u1))]
    X = np.array([(xi / xi[-1])[:3] for xi in X])
    return X


def _triangulate(P0,P1,x1,x2):
    # P0,P1: projection matrices for each of two cameras/images
    # x1,x1: corresponding points in each of two images (If using P that
    # has been scaled by K, then use camera coordinates, otherwise use
    # generalized coordinates)
    A = np.array([
        [P0[2,0]*x1[0] - P0[0,0], P0[2,1]*x1[0] - P0[0,1], P0[2,2]*x1[0] - P0[0,2], P0[2,3]*x1[0] - P0[0,3]],
        [P0[2,0]*x1[1] - P0[1,0], P0[2,1]*x1[1] - P0[1,1], P0[2,2]*x1[1] - P0[1,2], P0[2,3]*x1[1] - P0[1,3]],
        [P1[2,0]*x2[0] - P1[0,0], P1[2,1]*x2[0] - P1[0,1], P1[2,2]*x2[0] - P1[0,2], P1[2,3]*x2[0] - P1[0,3]],
        [P1[2,0]*x2[1] - P1[1,0], P1[2,1]*x2[1] - P1[1,1], P1[2,2]*x2[1] - P1[1,2], P1[2,3]*x2[1] - P1[1,3]]
    ])
    u,s,vt = np.linalg.svd(A)
    return vt[-1]


def get_focal_length(path, w):
    exif = piexif.load(path)
    return exif['Exif'][piexif.ExifIFD.FocalLengthIn35mmFilm] / 36 * w


ImgPair = namedtuple(
    "ImgPair",
    (
        "matches", "matched_kps", "u1", "u2", "ui1", "ui2",
         "x1", "x2", "inliers", "E", "K", "P_1", "P_2"
    )
)


SiftImage = namedtuple("SiftImage", ("img", "kp", "des"))


def compute_matches(des1, des2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    # Apply ratio test
    good = []
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            good.append(m)
    return good


class PointCloudGenerator:
    def __init__(self, imgs, f):
        self.f = f
        sift = cv2.xfeatures2d.SIFT_create()
        self.imgs = [SiftImage(i, *sift.detectAndCompute(i, None)) for i in imgs]
        self.pairs = []
        for i in range(len(self.imgs) - 1):
            p = process_img_pair(*self.imgs[i], *self.imgs[i+1], f)
            self.pairs.append(p)
        # Work in progress
#         self.common_kp = []
#         kp1 = set(self.pairs[i].matched_kp[:,2])
#         for i in range(self.imgs-1):
#             kp2 = set(self.pair[i+1].matched_kp[:,0])
#             self.common_kp.append(kp1.intersection(kp2))
        

def process_img_pair(img1, kp1, des1, img2, kp2, des2, f):
    h, w, _ = img1.shape

    good = compute_matches(des1, des2)
    u1 = []
    u2 = []
    matched_kps = []
    for m in good:
        matched_kps.append((kp1[m.queryIdx], des1[m.queryIdx], kp2[m.trainIdx], des2[m.trainIdx]))
        u1.append(kp1[m.queryIdx].pt)
        u2.append(kp2[m.trainIdx].pt)
    u1 = np.array(u1)
    u2 = np.array(u2)
    #Make homogeneous
    u1 = np.c_[u1, np.ones(u1.shape[0])]
    u2 = np.c_[u2, np.ones(u2.shape[0])]

    cu = w // 2
    cv = h // 2
    # Camera matrix
    K_cam = np.array([
        [f, 0, cu],
        [0, f, cv],
        [0, 0, 1]
    ])
    K_inv = np.linalg.inv(K_cam)
    x1 = u1 @ K_inv.T
    x2 = u2 @ K_inv.T
    # Compute essential matrix with RANSAC
    E, inliers = cv2.findEssentialMat(
        x1[:, :2], x2[:, :2], np.eye(3), method=cv2.RANSAC, threshold=1e-3
    )
    inliers = inliers.ravel().astype(bool)

    n_in, R, t, _ = cv2.recoverPose(E, x1[inliers, :2], x2[inliers, :2])
    # P_i = [R|t] with first image considered canonical
    P_1 = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0]
    ])
    P_2 = np.hstack((R, t))
    return ImgPair(
        good, matched_kps, u1, u2, u1[inliers], u2[inliers],
        x1, x2, inliers, E, K_cam, P_1, P_2
    )

In [18]:
ifnames = [
    "falcon01.jpg",
    "falcon02.jpg",
    "falcon03.jpg",
    "falcon04.jpg",
    "falcon05.jpg",
    "falcon06.jpg",
    "falcon07.jpg",
    "falcon08.jpg",
]
imgs = [plt.imread(f) for f in ifnames]

In [38]:
f = get_focal_length(ifnames[0], w)
pcg = PointCloudGenerator(imgs[:3], f)

In [ ]:
# OLD CODE BELOW
#
#
h, w, _ = imgs[0].shape
f = get_focal_length(ifnames[0], w)
pair12 = process_img_pair(*imgs[:2], f)
pair23 = process_img_pair(*imgs[1:3], f)

In [ ]:
P1c = pair12.K @ pair12.P_1
P2c = pair12.K @ pair12.P_2
P3 = np.vstack((pair12.P_2, [0, 0, 0, 1])) @ np.vstack((pair23.P_2, [0, 0, 0, 1]))
P3c = pair23.K @ P3[:-1]

X12 = triangulate(P1c, P2c, pair12.ui1, pair12.ui2)
X23 = triangulate(P2c, P3c, pair23.ui1, pair23.ui2)

In [ ]:
pcd23 = np.empty((X23.shape[0], 6))
pcd23[:, :3] = X23
pcd23[:, 3:] = extract_pixel_colors(imgs[1], pair23.ui1)
# plot_pcd(pcd23)

In [ ]:
m1 = X12.shape[0]
pcd = np.empty((m1 + X23.shape[0], 6))
pcd[:m1, :3] = X12
pcd[:m1, 3:] = extract_pixel_colors(imgs[0], pair12.ui1)
pcd[m1:, :] = pcd23

# plot_pcd(pcd)